In [20]:
#### CONFIGURATION OF PUBLIC CLUSTER (public, administration, services) ####

In [21]:
import topogenesis as tg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
# create an array of 0s
# dimensions are selected from the vacant area that was selected 
# during the urban analysis
width = 98
length = 35
avail_array = np.zeros((width,length,1), dtype=int)
p_array = np.copy(avail_array)

In [ ]:
# create array every i cells
def grid(i):
    array = np.array(avail_array, copy=True)
    array[0::i,::i] = 1
    array[::i,i-1::i] = 2
    array[i-1::i,i-1::i] = 3
    array[i-1::i,::i] = 4
    return array

In [ ]:
# create 8 cell array
eight_array = grid(8)
# create 2 cell array
two_array = grid(2)

In [ ]:
def plot(array_x):
    fig, ax = plt.subplots()
    plt.imshow(array_x)
    for x in range(98):
        ax.axhline(x, lw=0.1, color='k', zorder=5)
    for x in range(35):
        ax.axvline(x, lw=0.1, color='k', zorder=5)
    plt.colorbar()
    plt.show()

In [ ]:
plot(eight_array)

In [ ]:
plot(two_array)

In [ ]:
first_column = eight_array[::,0]
a = np.argwhere(first_column ==1)
point = a[len(a)//2]

In [ ]:
def courtyard(point):
    c = (8,8,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = 1

In [ ]:
# applying main courtyard to the middle of the plot 
# connected to the smallest side with the road
courtyard(point)
point_2 = (point[0], point[1]+8, 1)
courtyard(point_2)
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])

In [ ]:
up_s = tg.create_stencil("moore", 2, 2)
up_s.function = tg.sfunc.sum 

In [ ]:
vn_stencil = tg.create_stencil("von_neumann", 1, 1)
vn_stencil[1, 1, 1] = 0
vn_stencil[1, 1, :] = 0
vn_stencil.function = tg.sfunc.sum 

In [ ]:
neigh2_lattice = avail_lattice.apply_stencil(up_s)

In [ ]:
plt.imshow((neigh2_lattice[:,:,0]>=1) * (avail_lattice[:,:,0]==0))
plt.colorbar()
plt.show()

In [ ]:
# apply corridor to all the sides of the main courtyard
corridor = np.argwhere((neigh2_lattice[:,:,0]>=1) * (avail_lattice[:,:,0]==0))
for i in range(len(corridor)):
    avail_array[corridor[i][0],corridor[i][1]] = 2

In [ ]:
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
neigh_lattice = avail_lattice.apply_stencil(vn_stencil)

In [ ]:
plt.imshow(neigh_lattice == 2)
plt.colorbar()
plt.show()

In [ ]:
def find_possible_p(new_lattice, array, avail, i):
    # points where we have neighbours and number 1s (down and side cells)
    points_2 = np.argwhere(((new_lattice == i)*(array == 1)*(avail == 0)) == 1)

    # points where we have neighbours and number 3s (up cells) - find 1s to apply spaces
    points_3 = np.argwhere(((new_lattice == i)*(array == 3)*(avail == 0)) == 1)
    point_A = points_3[:][:]-[7,7,0]
    neg = np.where(point_A[:][:] < 0)
    point_A = np.delete(point_A, neg, axis=0)

    # all points together
    p = np.append(points_2, point_A, axis=0)
    return p

In [ ]:
p = find_possible_p(neigh_lattice, two_array, avail_array, 2)

In [ ]:
# definition of public spaces with 8*8 m2 (top starting point, index number)
def public(point,i):
    c = (8,8,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i

In [ ]:
# array of possible points
p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(p)):
    p_array[p[i][0], p[i][1]] = 1
plot(p_array)

In [ ]:
# adding joker 8*8 
# connected to the corridor of the main courtyard
i = 0
while i <= 0:
    a = False
    while a == False:
        si = p[np.random.choice(len(p), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 4
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 4
        a = ar or ac
        if (a==True)*(avail_array[si[0], si[1]] != 3) == 1:
            public(si,3)
            i = i + 1
            if ar == True:
                p_array[si[0]:si[0]+8, si[1]] = 0
            else:
                p_array[si[0], si[1]:si[1]+8] = 0
        else:
            a = False
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
neigh_lattice = avail_lattice.apply_stencil(vn_stencil)

In [ ]:
p = find_possible_p(neigh_lattice, two_array, avail_array, 2)

In [ ]:
# array of possible points
p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(p)):
    p_array[p[i][0], p[i][1]] = 1
plot(p_array)

In [ ]:
# adding sitting area 8*8 
# connected to the corridor of the main courtyard
i = 0
while i <= 0:
    a = False
    while a == False:
        si = p[np.random.choice(len(p), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 4
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 4
        a = ar or ac
        if (a==True)*(avail_array[si[0], si[1]] != 3) == 1:
            public(si,10)
            i = i + 1
            if ar == True:
                p_array[si[0]:si[0]+8, si[1]] = 0
            else:
                p_array[si[0], si[1]:si[1]+8] = 0
        else:
            a = False
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
neigh_lattice = avail_lattice.apply_stencil(vn_stencil)

In [ ]:
p = find_possible_p(neigh_lattice, two_array, avail_array, 2)

In [ ]:
# array of possible points
p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(p)):
    p_array[p[i][0], p[i][1]] = 1
plot(p_array)

In [ ]:
# adding playground 8*8 
# connected to the corridor of the main courtyard
i = 0
while i <= 0:
    a = False
    while a == False:
        si = p[np.random.choice(len(p), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 4
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 4
        a = ar or ac
        if (a==True)*(avail_array[si[0], si[1]] != 3) == 1:
            public(si,11)
            i = i + 1
            if ar == True:
                p_array[si[0]:si[0]+8, si[1]] = 0
            else:
                p_array[si[0], si[1]:si[1]+8] = 0
        else:
            a = False
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
neigh_lattice = avail_lattice.apply_stencil(vn_stencil)

In [ ]:
# definition of secondary spaces with 8*4 m2 (top starting point, index number)
def services_v(point,i):
    c = (8,4,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i
def services_h(point,i):
    c = (4,8,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i
def services(point,i,h,v):
    if h == True:
        services_h(point,i)
    else:
        services_v(point,i)

In [ ]:
p = find_possible_p(neigh_lattice, two_array, avail_array, 2)

In [ ]:
# array of possible points
p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(p)):
    p_array[p[i][0], p[i][1]] = 1
plot(p_array)

In [ ]:
# adding public toilets & canteen unit 8*4 
# connected from the smallest edge to the corridor of the main courtyard
b = np.argwhere(p_array == 1)
i = 0
j = 0
while i <= 1:
    av = False
    while (av == False) and (j < 10000):
        bi = b[np.random.choice(len(b), 1, replace=False)[0]]
        ar = np.sum(p_array[bi[0]:bi[0]+4, bi[1]]==1) == 2
        ac = np.sum(p_array[bi[0], bi[1]:bi[1]+4]==1) == 2
        av = ar or ac
        if (av == True) * (avail_array[bi[0], bi[1]] == 0) == 1:
            services(bi,4,ar,ac)
            i = i + 1
            if ar == True:
                p_array[bi[0]:bi[0]+8, bi[1]] = 0
            else:
                p_array[bi[0], bi[1]:bi[1]+8] = 0
        else:
            av = False
            j = j + 1
    if j == 10000:
        i = i + 1
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==10)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

In [ ]:
# points where we have neighbours and number 1s (down and side cells)
pf = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

In [ ]:
# array of possible points
p_array = np.zeros((width,length,1), dtype=int)

for i in range(len(pf)):
    p_array[pf[i][0], pf[i][1]] = 1
plot(p_array)

In [ ]:
plt.imshow((neigh_lattice==1))
plt.colorbar()
plt.show()
si = pf[np.random.choice(len(pf), 1, replace=False)[0]]

In [ ]:
# adding First_aid 8*8 connected to Sitting Area
i = 0
j = 0
while i <= 0:
    a = False
    while (a == False) and (j < 5000):
        si = pf[np.random.choice(len(pf), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 8
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 8
        a = ar or ac
        if a==True:
            if ar == True:
                if avail_array[si[0],si[1]-1] == 0:
                    if np.sum(avail_array[si[0],si[1]:si[1]-8] == 0) == 8:
                        si = si[:][:]-[0,7,0]      
                    else:
                        a = False  
                else:
                    if np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) != 8 :
                        a = False         
            else:
                if avail_array[si[0]-1,si[1]] == 0:
                    if np.sum(avail_array[si[0]-8,si[1]] == 0) == 8:
                        si = si[:][:]-[7,0,0]  
                    else:
                        a = False
                else:
                   if np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) != 8:
                        a = False 
            if a == True:
                public(si,5)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 5000:
        i = i + 1
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==3)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

In [ ]:
# points where we have neighbours and number 1s (down and side cells)
pf = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

In [ ]:
# array of possible points
p_array = np.zeros((width,length,1), dtype=int)

for i in range(len(pf)):
    p_array[pf[i][0], pf[i][1]] = 1
plot(p_array)

In [ ]:
plt.imshow((neigh_lattice==1))
plt.colorbar()
plt.show()
si = pf[np.random.choice(len(pf), 1, replace=False)[0]]

In [ ]:
# adding Joker expansion 8*8 connected to first Joker
i = 0
j = 0
while i <= 0:
    a = False
    while (a == False) and (j < 1000):
        si = pf[np.random.choice(len(pf), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 8
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 8
        a = ar or ac
        if a==True:
            if ar == True:
                if avail_array[si[0],si[1]-1] == 0:
                    if np.sum(avail_array[si[0],si[1]:si[1]-8] == 0) == 8:
                        si = si[:][:]-[0,7,0]      
                    else:
                        a = False  
                else:
                    if np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) != 8 :
                        a = False         
            else:
                if avail_array[si[0]-1,si[1]] == 0:
                    if np.sum(avail_array[si[0],si[1]:si[1]-8] == 0) == 8:
                        si = si[:][:]-[7,0,0]  
                    else:
                        a = False
                else:
                   if np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) != 8:
                        a = False 
            if a == True:
                public(si,3)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 1000:
        i = i + 1
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==4)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

plt.imshow(neigh_lattice == 1)
plt.colorbar()
plt.show()

In [ ]:
# array of possible points
pk = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pk)):
    p_array[pk[i][0], pk[i][1]] = 1

plt.imshow(p_array == 1)
plt.colorbar()
plt.show()

In [ ]:
# adding 1 Kitchen 8*8 connected to the Canteen 
i = 0
j = 0
while i <= 0:
    a = False
    while (a == False) and (j < 50000):
        si = pk[np.random.choice(len(pk), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 8
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 8
        a = ar or ac
        if (a==True) and (p_array[si[0],si[1]]==1):
            if ar == True:
                p_array[si[0]:si[0]+7, si[1]] = 0
                if avail_array[si[0],si[1]-1] == 0:
                    if (avail_array[si[0]+7,si[1]-7] != 0) or (avail_array[si[0]-7,si[1]+7] != 0):
                        a = False 
                    else:
                        si = si[:][:]-[0,7,0] 
                else:
                    if (avail_array[si[0]-7,si[1]+7] != 0) or (avail_array[si[0]+7,si[1]+7] != 0):
                        a = False         
            else:
                p_array[si[0], si[1]:si[1]+7] = 0
                if avail_array[si[0]-1,si[1]] == 0:
                    if (avail_array[si[0]-7,si[1]+7] != 0) or (avail_array[si[0]-7,si[1]+7] != 0):
                        a = False 
                    else:
                        si = si[:][:]-[7,0,0] 
                else:
                   if (avail_array[si[0]+7,si[1]+7] != 0) or (avail_array[si[0]+7,si[1]-7] != 0):
                        a = False 
            if a == True:
                public(si,6)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 50000:
        i = i + 1
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==6)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

In [ ]:
# array of possible points
pch = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pch)):
    p_array[pch[i][0], pch[i][1]] = 1

plt.imshow(neigh_lattice == 1)
plt.colorbar()
plt.show()

In [ ]:
# adding 1 Changing room 8*4 connected to the kitchen 
i = 0
j = 0
while i <= 0:
    a = False
    while (a == False) and (j < 5000):
        si = pch[np.random.choice(len(pch), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 8
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 8
        a = ar or ac
        if (a==True) and (p_array[si[0],si[1]]==1):
            if ar == True:
                p_array[si[0]:si[0]+7, si[1]] = 0
                if avail_array[si[0],si[1]-1] == 0:
                    if (avail_array[si[0]+1,si[1]-1] == 3) or (avail_array[si[0]+1,si[1]-1] == 1):
                        a = False 
                    else:
                        si = si[:][:]-[0,3,0] 
                else:
                    if (avail_array[si[0]+1,si[1]+1] == 3) or (avail_array[si[0]+1,si[1]+1] == 1):
                        a = False         
            else:
                p_array[si[0], si[1]:si[1]+7] = 0
                if avail_array[si[0]-1,si[1]] == 0:
                    if (avail_array[si[0]-1,si[1]+1] == 3) or (avail_array[si[0]-1,si[1]+1] == 1):
                        a = False 
                    else:
                        si = si[:][:]-[3,0,0] 
                else:
                   if (avail_array[si[0]+1,si[1]+1] == 3) or (avail_array[si[0]+1,si[1]+1] == 1):
                        a = False 
            if a == True:
                services(si,7,ac,ar)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 5000:
        i = i + 1
plot(avail_array)

In [ ]:
# changing room OLD_VERSION
# i = 0
# j = 0
# while i <= 0:
#     a = False
#     while (a == False) and (j < 500):
#         si = pch[np.random.choice(len(pch), 1, replace=False)[0]]
#         ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 8
#         ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 8
#         a = ar or ac
#         if a==True:
#             if ar == True:
#                 #p_array[si[0]:si[0]+7, si[1]] = 0
#                 if avail_array[si[0],si[1]-1] == 0:
#                     if np.sum(avail_array[si[0],si[1]:si[1]-8] == 0) == 8:
#                         si = si[:][:]-[0,7,0]      
#                     else:
#                         a = False  
#                 else:
#                     if np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) != 8 :
#                         a = False         
#             else:
#                 #p_array[si[0], si[1]:si[1]+8] = 0
#                 if avail_array[si[0]-1,si[1]] == 0:
#                     if np.sum(avail_array[si[0]-8,si[1]] == 0) == 8:
#                         si = si[:][:]-[7,0,0]  
#                     else:
#                         a = False
#                 else:
#                    if np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) != 8:
#                         a = False 
#             if a == True:
#                 services(si,7,ac,ar)
#                 i = i + 1
#         else:
#             a = False
#             j = j + 1
#     if j == 500:
#         i = i + 1
# plot(avail_array)

In [ ]:
only_up_stencil = tg.create_stencil("von_neumann", 1, 1)
only_up_stencil[1, 1, 1] = 0
only_up_stencil[1, 1, :] = 0
only_up_stencil[0, :, :] = 0
only_up_stencil[1, :, :] = 0
only_up_stencil.function = tg.sfunc.sum 

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==2)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(only_up_stencil)

In [ ]:
plt.imshow(neigh_lattice == 1)
plt.colorbar()
plt.show()

In [ ]:
# array of possible points
pch = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pch)):
    p_array[pch[i][0], pch[i][1]] = 1

plt.imshow(p_array == 1)
plt.colorbar()
plt.show()

In [ ]:
# definition of corridor with 8*2 m2 (top starting point, index number)
def corrid_v(point,i):
    c = (16,2,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i
def corrid_h(point,i):
    c = (2,16,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i
def corrid(point,i,h,v):
    if h == True:
        corrid_v(point,i)
    else:
        corrid_h(point,i)

In [ ]:
# adding 1 corridors 8*2 for distribution to the classrooms (only up)
a = False
while a == False:
    si = pch[np.random.choice(len(pch), 1, replace=False)[0]]
    ac = np.sum(p_array[si[0], si[1]:si[1]+2]==1) == 2
    if ac == True:
        if np.sum(avail_array[si[0],si[1]-7 == 0] == 7) == 0:
            si = si[:][:]-[15,0,0]
            corrid(si,8,ac,ar)
            a = True
        else:
            a = False
    else:
        a = False
plot(avail_array)

In [ ]:
only_down_stencil = tg.create_stencil("von_neumann", 1, 1)
only_down_stencil[1, 1, 1] = 0
only_down_stencil[1, 1, :] = 0
only_down_stencil[2, :, :] = 0
only_down_stencil[1, :, :] = 0
only_down_stencil.function = tg.sfunc.sum

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==2)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(only_down_stencil)

In [ ]:
# array of possible points
pch = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pch)):
    p_array[pch[i][0], pch[i][1]] = 1

plt.imshow(p_array == 1)
plt.colorbar()
plt.show()

In [ ]:
# adding 1 corridors 8*2 for distribution to the classrooms (only down)
a = False
while a == False:
    si = pch[np.random.choice(len(pch), 1, replace=False)[0]]
    ac = np.sum(p_array[si[0], si[1]:si[1]+2]==1) == 2
    if ac == True:
        if np.sum(avail_array[si[0],si[1]+7 == 0] == 7) == 0:
            corrid(si,8,ac,ar)
            a = True
        else:
            a = False
    else:
        a = False
plot(avail_array)

In [ ]:
# corridor OLD_VERSION
# i = 0
# j = 0
# while i <= 1:
#     a = False
#     while (a == False) and (j < 5000):
#         si = pch[np.random.choice(len(pch), 1, replace=False)[0]]
#         ar = np.sum(p_array[si[0]:si[0]+2, si[1]]==1) == 2
#         ac = np.sum(p_array[si[0], si[1]:si[1]+2]==1) == 2
#         a = ar or ac
#         if a==True:
#             if ar == True:
#                 if avail_array[si[0],si[1]-1] == 0:
#                     if np.sum(avail_array[si[0],si[1]:si[1]-8] == 0) == 8:
#                         si = si[:][:]-[0,15,0]      
#                     else:
#                         a = False  
#                 else:
#                     if np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) != 8 :
#                         a = False         
#             else:
#                 if avail_array[si[0]-1,si[1]] == 0:
#                     if np.sum(avail_array[si[0]:si[0]-8,si[1]] == 0) == 8:
#                         si = si[:][:]-[15,0,0]  
#                     else:
#                         a = False
#                 else:
#                    if np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) != 8:
#                         a = False 
#             if a == True:
#                 corrid(si,8,ac,ar)
#                 i = i + 1
#         else:
#             a = False
#             j = j + 1
#     if j == 5000:
#         i = i + 1
# plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==8)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

In [ ]:
# array of possible points
pcor = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pcor)):
    p_array[pcor[i][0], pcor[i][1]] = 1

plt.imshow(p_array == 1)
plt.colorbar()
plt.show()

In [ ]:
# adding 3 administration rooms 8*4 connected to the corridor
i = 0
j = 0
while i <= 2:
    a = False
    while (a == False) and (j < 9000):
        si = pcor[np.random.choice(len(pcor), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+4, si[1]]==1) == 4
        ac = np.sum(p_array[si[0], si[1]:si[1]+4]==1) == 4
        a = ar or ac
        if a==True:
            if ar == True:
                if avail_array[si[0],si[1]-1] == 0:
                    if (np.sum(avail_array[si[0],si[1]:si[1]-8] == 0) == 8) and (np.sum(avail_array[si[0]+4,si[1]:si[1]-8] == 0) == 8):
                        si = si[:][:]-[0,7,0]      
                    else:
                        a = False  
                else:
                    if (np.sum(avail_array[si[0],si[1]:si[1]+8] == 0) == 8) and (np.sum(avail_array[si[0]+4,si[1]:si[1]+8] == 0) == 8):
                        a = True
                    else:
                        a = False         
            if a == True:
                services(si,9,ar,ac)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 9000:
        i = i + 1
plot(avail_array)

In [ ]:
up_stencil = tg.create_stencil("von_neumann", 1, 1)
up_stencil[1, 1, 1] = 0
up_stencil[1, 1, :] = 0
up_stencil[1, :, 1] = 0
up_stencil.function = tg.sfunc.sum 

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==8)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(up_stencil)

In [ ]:
# array of possible points
pcor = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pcor)):
    p_array[pcor[i][0], pcor[i][1]] = 1

plt.imshow(p_array == 1)
plt.colorbar()
plt.show()

In [ ]:
# adding one courtyard for classrooms (above)
si = np.argwhere(p_array == 1)
si = si[0]
if np.ndarray.item(avail_array[si[0]-1, si[1]]) == 0:
    if np.sum(avail_array[si[0]-3,si[1]:si[1]+8] == 0) == 8:
        si = si[:][:]-[9,-2,0]
        public(si,12)
else:
    if np.sum(avail_array[si[0]+3,si[1]:si[1]+8] == 0) == 8:
        si = si[:][:]-[-2,-2,0]
        public(si,12)
plot(avail_array)

In [ ]:
# adding another courtyard for classrooms (below)
si = np.argwhere(p_array == 1)
si = si[3]
if np.ndarray.item(avail_array[si[0]-1, si[1]]) == 0:
    if np.sum(avail_array[si[0]-3,si[1]:si[1]+8] == 0) == 8:
        si = si[:][:]-[9,-2,0]
        public(si,12)
else:
    if np.sum(avail_array[si[0]+3,si[1]:si[1]+8] == 0) == 8:
        si = si[:][:]-[-2,-1,0]
        public(si,12)
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==12)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(up_s)
plot(neigh_lattice)

In [ ]:
# apply corridors around courtyards for classrooms 
corridor = np.argwhere((neigh_lattice[:,:,0]>=1) * (avail_lattice[:,:,0]==0))
for i in range(len(corridor)):
    avail_array[corridor[i][0],corridor[i][1]] = 13

In [ ]:
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==13)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

In [ ]:
# definition of classroom with 8*6 m2 (top starting point, index number)
def classroom_v(point,i):
    c = (8,6,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i
def classroom_h(point,i):
    c = (6,8,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i
def classroom(point,i,h,v):
    if h == True:
        classroom_h(point,i)
    else:
        classroom_v(point,i)

In [ ]:
# array of possible points
pcl = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)
print(len(pcl))

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pcl)):
    p_array[pcl[i][0], pcl[i][1]] = 1

plot(p_array)

In [ ]:
# adding 4 classrooms 8*6 around corridors
i = 0
j = 0
while i <= 3:
    a = False
    while (a == False) and (j < 9000):
        si = pcl[np.random.choice(len(pcl), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 8
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 8
        a = ar or ac
        if (a==True) and (p_array[si[0],si[1]]==1):
            if ar == True:
                p_array[si[0]:si[0]+7, si[1]] = 0
                if avail_array[si[0],si[1]-1] == 0:
                    if avail_array[si[0], si[1]:si[1]-6] == 0:
                        si = si[:][:]-[0,5,0] 
                    else:
                        a = False
                else:
                    if avail_array[si[0], si[1]:si[1]+6] == 0:
                        si = si[:][:]-[0,5,0] 
                    else:
                        a = False 
            else:
                p_array[si[0], si[1]:si[1]+7] = 0
                if avail_array[si[0]-1,si[1]] == 0:
                    if avail_array[si[0]:si[0]-6, si[1]] == 0:
                        si = si[:][:]-[5,0,0]
                    else:
                        a = False
                else:
                    if avail_array[si[0]:si[0]+6, si[1]] == 0:
                        si = si[:][:]-[0,5,0] 
                    else:
                        a = False                    
            if a == True:
                classroom(si,14,ac,ar)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 9000:
        i = i + 1
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==12)] = 1
k_array[np.where(avail_lattice==14)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

In [ ]:
# array of possible points
pc1 = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)
pc2 = np.argwhere(((neigh_lattice == 1)*(avail_array == 13)) == 1)
pc = np.append(pc1, pc2, axis=0)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pc)):
    p_array[pc[i][0], pc[i][1]] = 1

plot(p_array)

In [ ]:
# adding 4 more courtyards 8*8 for more classrooms
i = 0
j = 0
while i <= 3:
    a = False
    while (a == False) and (j < 10000):
        si = pc[np.random.choice(len(pc), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 8
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 8
        a = ar or ac
        if (a==True) and (p_array[si[0],si[1]]==1):
            if ar == True:
                p_array[si[0]:si[0]+7, si[1]] = 0
                if avail_array[si[0],si[1]-1] == 0:
                    if avail_array[si[0]+3,si[1]-3] != 0:
                        a = False 
                    else:
                        si = si[:][:]-[0,7,0] 
                else:
                    if avail_array[si[0]+3,si[1]+3] != 0:
                        a = False         
            else:
                p_array[si[0], si[1]:si[1]+7] = 0
                if avail_array[si[0]-1,si[1]] == 0:
                    if avail_array[si[0]-3,si[1]+3] != 0:
                        a = False 
                    else:
                        si = si[:][:]-[7,0,0] 
                else:
                   if avail_array[si[0]+3,si[1]+3] != 0:
                        a = False 
            if a == True:
                public(si,12)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 10000:
        i = i + 1
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==12)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(up_s)

In [ ]:
# apply corridor 
corridor = np.argwhere((neigh_lattice >= 1) * (avail_array==0))
for i in range(len(corridor)):
    avail_array[corridor[i][0],corridor[i][1]] = 13

plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==13)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

In [ ]:
pcl2= np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(pcl2)):
    p_array[pcl2[i][0], pcl2[i][1]] = 1

In [ ]:
# classroom vol2
i = 0
j = 0
while i <= 5:
    a = False
    while (a == False) and (j < 800):
        si = pcl2[np.random.choice(len(pcl2), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+8, si[1]]==1) == 8
        ac = np.sum(p_array[si[0], si[1]:si[1]+8]==1) == 8
        a = ar or ac
        if (a==True) and (p_array[si[0],si[1]]==1):
            if ar == True:
                p_array[si[0]:si[0]+7, si[1]] = 0
                if avail_array[si[0],si[1]-1] == 0:
                    if (avail_array[si[0]+4,si[1]-6] != 0) or (avail_array[si[0]-4,si[1]-6] != 0):
                        a = False 
                    else:
                        si = si[:][:]-[0,5,0] 
                else:
                    if (avail_array[si[0]+4,si[1]+6] != 0) or (avail_array[si[0]-4,si[1]+6] != 0):
                        a = False         
            else:
                p_array[si[0], si[1]:si[1]+7] = 0
                if avail_array[si[0]-1,si[1]] == 0:
                    if (avail_array[si[0]-6,si[1]+4] != 0) or (avail_array[si[0]-6,si[1]-4] != 0):
                        a = False 
                    else:
                        si = si[:][:]-[5,0,0] 
                else:
                   if (avail_array[si[0]+6,si[1]+4] != 0) or (avail_array[si[0]+6,si[1]-4] != 0):
                        a = False 
            if a == True:
                classroom(si,15,ac,ar)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 800:
        i = i + 1
plot(avail_array)

In [ ]:
avail_lattice = tg.to_lattice(avail_array, minbound=[0,0,0])
k_array = np.zeros((width,length,1), dtype=int)
k_array[np.where(avail_lattice==13)] = 1
k_lattice = tg.to_lattice(k_array, minbound=[0,0,0])
neigh_lattice = k_lattice.apply_stencil(vn_stencil)

ptl1 = np.argwhere(((neigh_lattice == 1)*(avail_array == 0)) == 1)
ptl2 = np.argwhere(((neigh_lattice == 1)*(avail_array == 13)) == 1)
ptl = np.append(ptl1, ptl2, axis=0)

p_array = np.zeros((width,length,1), dtype=int)
for i in range(len(ptl)):
    p_array[ptl[i][0], ptl[i][1]] = 1

plot(p_array)

In [ ]:
# definition toilet
def services_v(point,i):
    c = (6,4,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i
def services_h(point,i):
    c = (4,6,1)
    avail_array[point[0]:point[0] + c[0], point[1]:point[1] + c[1]] = i
def services(point,i,h,v):
    if h == True:
        services_h(point,i)
    else:
        services_v(point,i)

In [ ]:
# adding toilets and storage rooms 6*4
# connected to classroom's corridors
i = 0
j = 0
while i <= 4:
    a = False
    while (a == False) and (j < 1200):
        si = ptl[np.random.choice(len(ptl), 1, replace=False)[0]]
        ar = np.sum(p_array[si[0]:si[0]+4, si[1]]==1) == 4
        ac = np.sum(p_array[si[0], si[1]:si[1]+4]==1) == 4
        a = ar or ac
        if (a==True) and (p_array[si[0],si[1]]==1):
            if ar == True:
                p_array[si[0]:si[0]+3, si[1]] = 0
                if avail_array[si[0],si[1]-1] == 0:
                    if avail_array[si[0]+1,si[1]-5] != 0:
                        a = False 
                    else:
                        si = si[:][:]-[0,5,0] 
                else:
                    if avail_array[si[0]+1,si[1]+5] != 0:
                        a = False         
            else:
                p_array[si[0], si[1]:si[1]+3] = 0
                if avail_array[si[0]-1,si[1]] == 0:
                    if avail_array[si[0]-5,si[1]+1] != 0:
                        a = False 
                    else:
                        si = si[:][:]-[5,0,0] 
                else:
                   if avail_array[si[0]+5,si[1]+1] != 0:
                        a = False 
            if a == True:
                services(si,4,ar,ac)
                i = i + 1
        else:
            a = False
            j = j + 1
    if j == 1200:
        i = i + 1
plot(avail_array)

In [ ]:
import pyvista as pv
from compas.datastructures import Mesh as CompasMesh
import compas
import pandas as pd

import earthy_bridge as eb

In [ ]:
newarr = np.reshape(avail_array, (98,35))

In [ ]:
# export to an excel
df = pd.DataFrame(newarr)
df.to_csv(r'C:\Users\Mariana\Desktop\export_dataframe.csv', index = False)